In [5]:
import numpy as np
import torch
import torch.nn.functional as F

In [11]:
np.array([[1,2,3],[2,3,4]]).max(0)

array([2, 3, 4])

In [6]:
torch.tensor([[1,2,3,4],[5,6,7,8]]).max(1) #第一个参数是value,第二个是index

(tensor([4, 8]), tensor([3, 3]))

In [7]:
logits = torch.rand(4,10)
pred = F.softmax(logits,dim=1) #遍历10列
pred.shape

torch.Size([4, 10])

In [8]:
pred

tensor([[0.1294, 0.1093, 0.0838, 0.1191, 0.1242, 0.0862, 0.0510, 0.1318, 0.0999,
         0.0654],
        [0.0584, 0.1214, 0.0678, 0.1074, 0.1504, 0.0891, 0.1115, 0.0621, 0.1416,
         0.0903],
        [0.0972, 0.1427, 0.0639, 0.1086, 0.0730, 0.0643, 0.0732, 0.1040, 0.1500,
         0.1231],
        [0.0620, 0.0739, 0.1487, 0.0677, 0.1005, 0.1242, 0.1064, 0.1351, 0.1219,
         0.0596]])

In [9]:
pred_label = pred.argmax(dim=1) #遍历10列的最大值
pred_label

tensor([7, 4, 8, 2])

In [12]:
label = torch.tensor([7,4,3,2])
correct = torch.eq(pred_label,label)

In [15]:
correct

tensor([1, 1, 0, 1], dtype=torch.uint8)

In [17]:
correct.sum().float().item()/4

0.75

In [21]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from    torchvision import datasets, transforms

from visdom import Visdom #这个工具需要开启 命令行python -m visdom.server，在浏览器中查看

batch_size=200
learning_rate=0.01
epochs=10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnistdata', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       # transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnistdata', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        # transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)



class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

viz = Visdom()

viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='test loss&acc.',
                                                   legend=['loss', 'acc.']))
global_step = 0

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.cuda()

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        global_step += 1
        viz.line([loss.item()], [global_step], win='train_loss', update='append')

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()

    viz.line([[test_loss, correct / len(test_loader.dataset)]],
             [global_step], win='test', update='append')
    viz.images(data.view(-1, 1, 28, 28), win='x')
    viz.text(str(pred.detach().cpu().numpy()), win='pred',
             opts=dict(title='pred'))

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.301537
Train Epoch: 0 [20000/60000 (33%)]	Loss: 2.271392
Train Epoch: 0 [40000/60000 (67%)]	Loss: 2.223447

Test set: Average loss: 0.0109, Accuracy: 5504.0/10000 (55%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.189170
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.096236
Train Epoch: 1 [40000/60000 (67%)]	Loss: 1.966659

Test set: Average loss: 0.0089, Accuracy: 5629.0/10000 (56%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.793783
Train Epoch: 2 [20000/60000 (33%)]	Loss: 1.673108
Train Epoch: 2 [40000/60000 (67%)]	Loss: 1.528649

Test set: Average loss: 0.0066, Accuracy: 6210.0/10000 (62%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.353001
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.244355
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.155593

Test set: Average loss: 0.0050, Accuracy: 7333.0/10000 (73%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.973035
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.898465
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.940837

Test set: Average loss

Connection is already closed.
Connection is already closed.
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
[Errno 99] Cannot assign requested address
